# This Notebook is the project deliverable for my completion of the <span style="text-decoration: underline"> Coursera Applied Data Science </span> course. 

Included in this Notebook are:
1. Capstone Project - The Battle of Neighborhoods - Week 1
2. Capstone Project - The Battle of Neighborhoods - Week 2

## This section is <span style="text-decoration: underline">The Battle of Neighborhoods - Week 1 </span>
    
For this step of the assignment I'm using data from https://data.census.gov , https://nces.ed.gov/ccd , and https://foursquare.com .


## The problem presented is: How to select an elementary school in Park City, Utah using data available for the schools and neighborhoods. ## 
#### The challenge in selecting a neighborhood based on school performance and offerings is common to parents moving to new homes in unfamiliar areas. This project provides a method for scoring schools and neighborhoods using publicly available data from government sources. ####

## The data used in the project is primarily from US government sites including NCES.ED and the US Census. ## 
The data from https://nces.ed.gov includes details such as the school level (Elementary, Middle, High), type (regular or special ed), indicator for charter schools, the TitleI status, dual language offerings, etc.   
Additional data may be used to indicate crime statistics but I have not yet located such data for Park City.   
Finally, https:foursquare.com is used to identify additional offerings or detriments proximal to the various schools and neighborhoods.


### End of Capstone Project - The Battle of Neighborhoods - Week 1

## This section is <span style="text-decoration: underline">The Battle of Neighborhoods - Week 2 </span>

## Methodology: ##

The methodology of this project includes:
1. Identifying, importing, and wrangling the required data
2. Use the data to observe factors for the subject schools
3. Review the school offerings
4. Establish relationships between school levels
5. Explore social options for parents

In [1]:
# import the libraries required by the analysis
import pandas as pd
import numpy as np
import folium
from folium import plugins
from folium.features import Choropleth
import geocoder
from geopy.geocoders import Nominatim
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import requests
from sklearn.cluster import KMeans

In [2]:
pd.options.display.max_columns = None     # make sure all dataframe columns can be inspected when peeking
pd.options.display.max_rows = None        # make sure all dataframe rows can be inspected when peeking

In [4]:
# start with some school related data in ccd_sch_029_1718_w_1a_083118.csv and ccd_sch_129_1718_w_1a_083118.csv
raw=pd.read_csv(r'C:\Education\Files\ccd_sch_029_1718_w_1a_083118.csv',low_memory=False)
raw2=pd.read_csv(r'C:\Education\Files\ccd_sch_129_1718_w_1a_083118.csv',low_memory=False)

In [ ]:
raw2.head()

In [5]:
df1=raw.loc[raw['MCITY']=='PARK CITY']                        # get all the park city school info
df2=raw2.loc[raw2['STATENAME']=='UTAH']                       # get additional school info

In [6]:
df1.reset_index(drop=True,inplace=True)
df2.reset_index(drop=True,inplace=True)

In [8]:
cols_to_use = df2.columns.difference(df1.columns)   # get coulmns to merge to remove duplicate columns
df1 = pd.merge(df1, df2[cols_to_use], left_index=True, right_index=True, how='outer')       # combine the dataframes without duplicates
df1.reset_index(drop=True,inplace=True)
df1=df1.head(14)                                            # clean up the junk from the merge
lookup = ['Weilenmann', 'Winter Sports School','Jeremy Ranch School','Park City High School','Ecker Hill School','Parleys Park Elementary School','Trailside School','Treasure Mountain School','McPolin School','McPolin School','McPolin School','Parleys Park Elementary School','Trailside School','Jeremy Ranch School']
df1['LOOKUP'] = lookup                                      # set up a column that geopy recognizes for coordinates
df1                                                        # look at the results

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT,LOOKUP
0,2017-2018,49.0,UTAH,UT,Weilenmann School of Discovery,Weilenmann School of Discovery,1.0,NaN,UT-6G,4900145.0,UT-6G-6G100,4.900145e+11,4901378.0,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098.0,NaN,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)575-5411,http://wsdpc.org/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,Yes,99,USBE,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,KG,08,Elementary,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,NOTTITLE1ELIG,Not a Title I school,NOTVIRTUAL,Not Virtual,Weilenmann
1,2017-2018,49.0,UTAH,UT,Winter Sports School,Winter Sports School,1.0,NaN,UT-8I,4900168.0,UT-8I-8I700,4.900168e+11,4901444.0,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098.0,NaN,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)649-8760,http://www.wintersportsschool.org,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,Yes,99,USBE,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,09,12,High,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,SWELIGSWPROG,Title I schoolwide school,NOTVIRTUAL,Not Virtual,Winter Sports School
2,2017-2018,49.0,UTAH,UT,Jeremy Ranch School,Park City District,1.0,NaN,UT-22,4900750.0,UT-22-22112,4.900750e+11,4900197.0,3050 RASMUSSEN RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,3050 RASMUSSEN RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)645-5670,http://jres.pcschools.us/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,No,NaN,NaN,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,KG,05,Elementary,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,SWELIGSWPROG,Title I schoolwide school,NOTVIRTUAL,Not Virtual,Jeremy Ranch School
3,2017-2018,49.0,UTAH,UT,Park City High,Park City District,1.0,NaN,UT-22,4900750.0,UT-22-22704,4.900750e+11,4900447.0,1750 KEARNS BLVD,NaN,NaN,PARK CITY,UT,84060.0,NaN,1750 KEARNS BLVD,NaN,NaN,PARK CITY,UT,84060.0,NaN,(435)645-5650,http://pchs.parkcity.k12.ut.us/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,No,No,No,10,12,High,As reported,No,NSLPNO,No,No,NOTTITLE1ELIG,Not a Title I school,NOTVIRTUAL,Not Virtual,Park City High School
4,2017-2018,49.0,UTAH,UT,Ecker Hill Middle,Park City District,1.0,NaN,UT-22,4900750.0,UT-22-22308,4.900750e+11,4900485.0,2465 W KILBY RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,2465 W KILBY RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)645-5610,http://ehms.pcschools.us/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,06,07,Middle,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,NOTTITLE1ELIG,Not a Title I school,NOTVIRTUAL,Not Virtual,Ecker Hill School
5,2017-2018,49.0,UTAH,UT,Parleys Park School,Park City District,1.0,NaN,UT-22,4900750.0,UT-22-22104,4.900750e+11,4900726.0,4600 SILVERSPRINGS DRIVE,NaN,NaN,PARK CITY,UT,84068.0,NaN,4600 SILVERSPRINGS DR,NaN,NaN,PARK CITY,UT,84068.0,NaN,(435)645-5620,http://ppes.parkcity.k12.ut.us/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,No,NaN,NaN,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,

In [9]:
# now get the coordinates for the schools which are in df1.loc(['MSTREET1'],['MCITY'],['MSTATE'],['MZIP']) and add them to the dataframe
df1["Latitude"] = 0
df1["Longitude"] = 0
for i in range(1,14):
    name = str(df1.loc[i,'LOOKUP'] + ', ' + df1.loc[i,'MSTATE'])
    address = name
    geolocator = Nominatim(user_agent="pcschools_explorer0")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    df1.loc[i,'Latitude']=latitude
    df1.loc[i,'Longitude']=longitude
# Weilenmann School of Discovery doesn't return results in geolocators so plug in the coordinates from Google Earth:
df1.loc[0,'Latitude'] = 40.7520938 
df1.loc[0,'Longitude'] = -111.592570
# Trailside schools don't return the correct results in geolocators so plug in the coordinates from Google Earth:
df1.loc[6,'Latitude'] = 40.715401 
df1.loc[6,'Longitude'] = -111.503347
# Trailside schools don't return the correct results in geolocators so plug in the coordinates from Google Earth:
df1.loc[12,'Latitude'] = 40.715401 
df1.loc[12,'Longitude'] = -111.503347
# Treasure Mountain Junior High doesn't return the correct results in geolocators so plug in the coordinates from Google Earth:
df1.loc[7,'Latitude'] = 40.668798 
df1.loc[7,'Longitude'] = -111.490434
df1.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT,LOOKUP,Latitude,Longitude
0,2017-2018,49.0,UTAH,UT,Weilenmann School of Discovery,Weilenmann School of Discovery,1.0,NaN,UT-6G,4900145.0,UT-6G-6G100,4.900145e+11,4901378.0,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098.0,NaN,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)575-5411,http://wsdpc.org/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,Yes,99,USBE,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,KG,08,Elementary,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,NOTTITLE1ELIG,Not a Title I school,NOTVIRTUAL,Not Virtual,Weilenmann,40.752094,-111.592570
1,2017-2018,49.0,UTAH,UT,Winter Sports School,Winter Sports School,1.0,NaN,UT-8I,4900168.0,UT-8I-8I700,4.900168e+11,4901444.0,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098.0,NaN,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)649-8760,http://www.wintersportsschool.org,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,Yes,99,USBE,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,09,12,High,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,SWELIGSWPROG,Title I schoolwide school,NOTVIRTUAL,Not Virtual,Winter Sports School,40.691752,-111.543052
2,2017-2018,49.0,UTAH,UT,Jeremy Ranch School,Park City District,1.0,NaN,UT-22,4900750.0,UT-22-22112,4.900750e+11,4900197.0,3050 RASMUSSEN RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,3050 RASMUSSEN RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)645-5670,http://jres.pcschools.us/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,No,NaN,NaN,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,KG,05,Elementary,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,SWELIGSWPROG,Title I schoolwide school,NOTVIRTUAL,Not Virtual,Jeremy Ranch School,40.754114,-111.559369
3,2017-2018,49.0,UTAH,UT,Park City High,Park City District,1.0,NaN,UT-22,4900750.0,UT-22-22704,4.900750e+11,4900447.0,1750 KEARNS BLVD,NaN,NaN,PARK CITY,UT,84060.0,NaN,1750 KEARNS BLVD,NaN,NaN,PARK CITY,UT,84060.0,NaN,(435)645-5650,http://pchs.parkcity.k12.ut.us/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,No,No,No,10,12,High,As reported,No,NSLPNO,No,No,NOTTITLE1ELIG,Not a Title I school,NOTVIRTUAL,Not Virtual,Park City High School,40.667509,-111.496910
4,2017-2018,49.0,UTAH,UT,Ecker Hill Middle,Park City District,1.0,NaN,UT-22,4900750.0,UT-22-22308,4.900750e+11,4900485.0,2465 W KILBY RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,2465 W KILBY RD,NaN,NaN,PARK CITY,UT,84098.0,NaN,(435)645-5610,http://ehms.pcschools.us/,1.0,Open,1.0,Open,05/15/2018,Regular School,1.0,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,06,07,Middle,As reported,No,NSLPWOPRO,Yes participating without using any Provision ...,No,NOTTITLE1ELIG,Not a Title I school,NOTVIRTUAL,Not Virtual,Ecker Hill School,40.740781,-111.575202


In [12]:
# get the park city coordinates
address = 'Park City, UT'
geolocator = Nominatim(user_agent="parkcity_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Park City, UT are {}, {}.'.format(latitude, longitude))

# create map of the Park City area 
map_park_city = folium.Map(location=[latitude, longitude], zoom_start=12)

# add neighborhood and school markers to map
for lat, lng, school in zip(df1['Latitude'], df1['Longitude'], df1['SCH_NAME']):
    label = '{}, {}'.format(school,name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_park_city)  
    
map_park_city              # generate the image

The geograpical coordinates of Park City, UT are 40.6460635, -111.4979741.


## This section explores school offerings. ##

Parents may choose a neighborhood based on offerings that may be unique to a local school.

In [ ]:
# assess number of schools for a given garde range

grade_level_data = df1.groupby(['G_1_OFFERED','G_2_OFFERED','G_3_OFFERED','G_4_OFFERED','G_5_OFFERED','G_6_OFFERED']).size().to_frame(name='Count').reset_index()
grade_level_data

# identify number of schools with a dual language immersion program and the language offered

language_options = df1.groupby(['DUAL', 'French', 'Spanish'])
language_options

## This section invokes foursquare services to explore areas surrounding the schools. ##

Parents frequently use coffee shops after school drop-off as social venues so let's see which schools might have more nearby options for people new to the area.

In [13]:
# establish foursquare identity (this will be removed from delivered project)

CLIENT_ID = 'nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn' 
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' 
VERSION = '20181201'
search_query='Coffee'
radius=3000
LIMIT=8
url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

sdf=[]                       # build a list of dataframes for the coffee shops nearest each school

for i in range(0,13):
    latitude=df1.loc[i,'Latitude']
    longitude=df1.loc[i,'Longitude']
    results = requests.get(url).json()
    venues = results['response']['venues']
    dfs=pd.json_normalize(venues)         #
    filtered_columns = ['name', 'categories'] + [col for col in dfs.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dfs.loc[:, filtered_columns]
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns] 
    sdf.append(dataframe_filtered)
    del dfs
    del dataframe_filtered
    del venues
    del results
    

In [18]:
sdf[1]

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Campos Coffee Park City,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...","1385 Lowell Avenue, Suite AC-106",40.651871,-111.507899,"[{'label': 'display', 'lat': 40.65187140267549...",1058,84060,US,Park City,UT,United States,"[1385 Lowell Avenue, Suite AC-106, Park City, ...",585d45a09398ab59401f68d4
1,"Atticus Coffee, Books and Teahouse","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",738 Main St,40.647279,-111.497943,"[{'label': 'display', 'lat': 40.64727880553917...",135,84060,US,Park City,UT,United States,"[738 Main St, Park City, UT 84060, United States]",4c69afbe897b1b8d83e1af17
2,Park City Coffee Roaster,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",1255 Park Avenue,40.646286,-111.497989,"[{'label': 'display', 'lat': 40.64628602356112...",24,84060,US,Park City,UT,United States,"[1255 Park Avenue, Park City, UT 84060, United...",5027cea6e4b0f23b0200c7ea
3,Lucky Ones Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,40.650897,-111.503295,"[{'label': 'display', 'lat': 40.650897, 'lng':...",701,84060,US,Park City,UT,United States,"[Park City, UT 84060, United States]",5abbe16e0336932abcc8ad01
4,Bumps 'n Grinds coffee shack,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,40.651753,-111.507636,"[{'label': 'display', 'lat': 40.65175263823070...",1032,NaN,US,Park City,UT,United States,"[Park City, UT, United States]",4b9297e2f964a520c90734e3
5,Silver King Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",1409 Kearns Blvd,40.661939,-111.501772,"[{'label': 'display', 'lat': 40.66193949736373...",1796,84060,US,Park City,UT,United States,"[1409 Kearns Blvd, Park City, UT 84060, United...",4e220755d1647406320cfc26
6,coffee bean and tea leaf,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",NaN,40.658589,-111.505771,"[{'label': 'display', 'lat': 40.658589, 'lng':...",1541,NaN,US,Park City,UT,United States,"[Park City, UT, United States]",531394f9498e43d4cda71572
7,Copper Cup Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,40.661957,-111.509386,"[{'label': 'display', 'lat': 40.661957, 'lng':...",2014,NaN,US,Park City,UT,United States,"[Park City, UT, United States]",568b11ff498e1016859f0303


### Results: ###

The results show that the project inception should have included more research into available data rather than hoping/assuming it would exist in a publicly consumable format.

### Discussion: ###

While it is possible to use economic, crime, and education data to score school choices, the greatest challenge in the assessment is to obtain relevant and reliable data. Consider neighborhood designations, crime statistics available by area, and economic data on the same boiundaries, the data can certainly be used to help guide parents to make decisions in selecting new neighborhoods and schools. In this case the relevant data was just not available, or at least not freely available. That is a limiting factor. 

## Conclusion: ##

#### My conclusions from this report are that: #### 
1) the Foursquare developer service is not ideal for high speed requests as are generated within a program loop. This is based on the random results returned for each iteration of the loop used to collect information on nearby coffee shops and  
2) the geopy Nominatum geolocator service is highly sensitive to specific names resulting in incorrect results and lacks many common public locations thus requiring extensive validation of the results once mapped. 

Unfortunately, in the interest of time, I had to drop some of the planned exploration after investing too many hours in data wrangling just to find the data was unreliable. 

### End of Capstone Project - The Battle of Neighborhoods - Week 2
### End of Notebook